# Sequential Q-learning

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
import gym
import numpy as np

In [2]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [3]:
import gym

# Create the Cart-Pole game environment
#env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [4]:
env.reset()
for _ in range(10):
    # env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

state, action, reward, done, info: [-0.01977852 -0.15034484 -0.02802841  0.2722864 ] 0 1.0 False {}
state, action, reward, done, info: [-0.02278542  0.04516561 -0.02258268 -0.02910336] 1 1.0 False {}
state, action, reward, done, info: [-0.02188211  0.24060401 -0.02316475 -0.32882495] 1 1.0 False {}
state, action, reward, done, info: [-0.01707003  0.04581936 -0.02974125 -0.04353619] 0 1.0 False {}
state, action, reward, done, info: [-0.01615364  0.24135489 -0.03061197 -0.34545241] 1 1.0 False {}
state, action, reward, done, info: [-0.01132654  0.43689862 -0.03752102 -0.6476291 ] 1 1.0 False {}
state, action, reward, done, info: [-0.00258857  0.63252268 -0.0504736  -0.95188725] 1 1.0 False {}
state, action, reward, done, info: [ 0.01006188  0.82828621 -0.06951135 -1.25999152] 1 1.0 False {}
state, action, reward, done, info: [ 0.02662761  1.0242251  -0.09471118 -1.57360997] 1 1.0 False {}
state, action, reward, done, info: [ 0.04711211  1.22034044 -0.12618338 -1.89426804] 1 1.0 False {}


To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [5]:
# print(rewards[-20:])
# print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
# print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
# print(np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
# print(np.max(np.array(rewards)), np.min(np.array(rewards)))
# print(np.max(np.array(states)), np.min(np.array(states)))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [6]:
def model_input(state_size, lstm_size, batch_size=1):
    actions = tf.placeholder(tf.int32, [None], name='actions')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    labelQs = tf.placeholder(tf.float32, [None], name='labelQs')
        
    gru = tf.nn.rnn_cell.GRUCell(lstm_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return actions, states, targetQs, labelQs, cell, initial_state

In [7]:
# RNN generator or sequence generator
def generator(states, initial_state, cell, lstm_size, num_classes, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=num_classes)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [8]:
def model_loss(action_size, hidden_size, states, cell, initial_state, actions, targetQs, labelQs):
    actions_logits, final_state = generator(states=states, cell=cell, initial_state=initial_state, 
                                            lstm_size=hidden_size, num_classes=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    Qs = tf.reduce_max(actions_logits*actions_labels, axis=1)
    lossQtgt = tf.reduce_mean(tf.square(Qs - targetQs)) # next state, next action and nextQs
    lossQlbl = tf.reduce_mean(tf.square(Qs - labelQs)) # current state, action, and currentQs
    lossQtgt_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs, 
                                                                           labels=tf.nn.sigmoid(targetQs)))
    lossQlbl_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs,
                                                                           labels=tf.nn.sigmoid(labelQs)))
    loss = lossQtgt + lossQlbl + lossQtgt_sigm + lossQlbl_sigm
    return actions_logits, final_state, loss, lossQtgt, lossQlbl, lossQtgt_sigm, lossQlbl_sigm

In [9]:
def model_opt(loss, learning_rate):
    """
    Get optimization operations in order
    :param loss: Generator loss Tensor for action prediction
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    # # Optimize
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    # #opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=g_vars)

    #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
    grads = tf.gradients(loss, g_vars)
    opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, g_vars))

    return opt

In [10]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.actions, self.states, self.targetQs, self.labelQs, cell, self.initial_state = model_input(
            state_size=state_size, lstm_size=hidden_size)
        
        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.final_state, self.loss, self.lossQtgt, self.lossQlbl, self.lossQtgt_sigm, self.lossQlbl_sigm = model_loss(
            action_size=action_size, hidden_size=hidden_size, 
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, labelQs=self.labelQs, 
            cell=cell, initial_state=self.initial_state)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [11]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
# print('state:', np.array(states).shape[1], 
#       'action size: {}'.format((np.max(np.array(actions)) - np.min(np.array(actions)))+1))

In [13]:
# Training parameters
# Network parameters
state_size = 4                 # number of units for the input state/observation -- simulation
action_size = 2                # number of units for the output actions -- simulation
hidden_size = 64               # number of units in each Q-network hidden layer -- simulation
batch_size = 32                # number of samples in the memory/ experience as mini-batch size
learning_rate = 0.001          # learning rate for adam

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 2)


## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [15]:
state = env.reset()
for _ in range(batch_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done is True:
        state = env.reset()

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [16]:
memory.buffer[0]

[array([ 0.0345243 , -0.00959876,  0.04293128,  0.02900301]),
 1,
 array([ 0.03433232,  0.18488207,  0.04351134, -0.24983153]),
 1.0,
 0.0]

In [17]:
# states, rewards, actions

In [ ]:
# Now train with experiences
saver = tf.train.Saver() # save the trained model
rewards_list, loss_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_loss = deque(maxlen=batch_size)
    episode_reward = deque(maxlen=batch_size)
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        loss_batch = []
        lossQlbl_batch, lossQlbl_sigm_batch, lossQtgt_batch, lossQtgt_sigm_batch = [], [], [], []
        state = env.reset()
        initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]

        # Training steps/batches
        while True:
            # Testing
            action_logits, final_state = sess.run([model.actions_logits, model.final_state],
                                                  feed_dict = {model.states: state.reshape([1, -1]), 
                                                               model.initial_state: initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append([initial_state, final_state])
            total_reward += reward
            initial_state = final_state
            state = next_state

            # Training
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            rnn_states = memory.states
            initial_states = np.array([each[0] for each in rnn_states])
            final_states = np.array([each[1] for each in rnn_states])
            actions_logits = sess.run(model.actions_logits, 
                                      feed_dict = {model.states: states, 
                                                   model.initial_state: initial_states[0].reshape([1, -1])})
            labelQs = np.max(actions_logits, axis=1) # explore
            next_actions_logits = sess.run(model.actions_logits, 
                                           feed_dict = {model.states: next_states, 
                                                        model.initial_state: final_states[0].reshape([1, -1])})
            nextQs = np.max(next_actions_logits, axis=1) * (1-dones) # exploit
            targetQs = rewards + (0.99 * nextQs)
            loss, _, lossQlbl, lossQlbl_sigm, lossQtgt, lossQtgt_sigm = sess.run([model.loss, model.opt, 
                                                                                  model.lossQlbl, 
                                                                                  model.lossQlbl_sigm, 
                                                                                  model.lossQtgt, 
                                                                                  model.lossQtgt_sigm], 
                                            feed_dict = {model.states: states, 
                                                         model.actions: actions,
                                                         model.targetQs: targetQs,
                                                         
                                                         model.labelQs: labelQs,
                                                         model.initial_state: initial_states[0].reshape([1, -1])})
            loss_batch.append(loss)
            lossQlbl_batch.append(lossQlbl)
            lossQlbl_sigm_batch.append(lossQlbl_sigm)
            lossQtgt_batch.append(lossQtgt)
            lossQtgt_sigm_batch.append(lossQtgt_sigm)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode: {}'.format(ep),
              'meanReward: {:.4f}'.format(np.mean(episode_reward)),
              'meanLoss: {:.4f}'.format(np.mean(loss_batch)),
              'meanLossQlbl: {:.4f}'.format(np.mean(lossQlbl_batch)),
              'meanLossQlbl_sigm: {:.4f}'.format(np.mean(lossQlbl_sigm_batch)),
              'meanLossQtgt: {:.4f}'.format(np.mean(lossQtgt_batch)),
              'meanLossQtgt_sigm: {:.4f}'.format(np.mean(lossQtgt_sigm_batch)))
        rewards_list.append([ep, np.mean(episode_reward)])
        loss_list.append([ep, np.mean(loss_batch)])
        if(np.mean(episode_reward) >= 500):
            break
    
    saver.save(sess, 'checkpoints/model5.ckpt')

Episode: 0 meanReward: 8.0000 meanLoss: 2.4964 meanLossQlbl: 0.0702 meanLossQlbl_sigm: 0.6805 meanLossQtgt: 1.0969 meanLossQtgt_sigm: 0.6489
Episode: 1 meanReward: 8.5000 meanLoss: 2.5235 meanLossQlbl: 0.6727 meanLossQlbl_sigm: 0.6896 meanLossQtgt: 0.4811 meanLossQtgt_sigm: 0.6801
Episode: 2 meanReward: 8.6667 meanLoss: 4.3015 meanLossQlbl: 2.0428 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 0.8724 meanLossQtgt_sigm: 0.6931
Episode: 3 meanReward: 8.7500 meanLoss: 6.0796 meanLossQlbl: 3.1585 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 1.5363 meanLossQtgt_sigm: 0.6918
Episode: 4 meanReward: 9.0000 meanLoss: 7.2201 meanLossQlbl: 3.9690 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 1.8655 meanLossQtgt_sigm: 0.6926
Episode: 5 meanReward: 8.8333 meanLoss: 7.6302 meanLossQlbl: 4.2173 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 2.0279 meanLossQtgt_sigm: 0.6920
Episode: 6 meanReward: 8.8571 meanLoss: 7.1715 meanLossQlbl: 3.9283 meanLossQlbl_sigm: 0.6930 meanLossQtgt: 1.8586 meanLossQtgt_sigm: 0.6916
Episode: 7 me

Episode: 59 meanReward: 9.3438 meanLoss: 0.7034 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0620 meanLossQtgt: 0.5220 meanLossQtgt_sigm: 0.1194
Episode: 60 meanReward: 9.3750 meanLoss: 0.6408 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1006 meanLossQtgt: 0.4061 meanLossQtgt_sigm: 0.1342
Episode: 61 meanReward: 9.3438 meanLoss: 0.6298 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1203 meanLossQtgt: 0.3688 meanLossQtgt_sigm: 0.1407
Episode: 62 meanReward: 9.3438 meanLoss: 0.7013 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0875 meanLossQtgt: 0.4843 meanLossQtgt_sigm: 0.1295
Episode: 63 meanReward: 9.3438 meanLoss: 0.6899 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0748 meanLossQtgt: 0.4902 meanLossQtgt_sigm: 0.1249
Episode: 64 meanReward: 9.3438 meanLoss: 0.5796 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0798 meanLossQtgt: 0.3751 meanLossQtgt_sigm: 0.1246
Episode: 65 meanReward: 9.3438 meanLoss: 0.5218 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0959 meanLossQtgt: 0.2973 meanLossQtgt_sigm: 0.1285
Episod

Episode: 117 meanReward: 9.4688 meanLoss: 0.5337 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1361 meanLossQtgt: 0.2497 meanLossQtgt_sigm: 0.1479
Episode: 118 meanReward: 9.4062 meanLoss: 0.5715 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0779 meanLossQtgt: 0.3708 meanLossQtgt_sigm: 0.1229
Episode: 119 meanReward: 9.4062 meanLoss: 0.4397 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0886 meanLossQtgt: 0.2282 meanLossQtgt_sigm: 0.1229
Episode: 120 meanReward: 9.4062 meanLoss: 0.3824 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1185 meanLossQtgt: 0.1287 meanLossQtgt_sigm: 0.1352
Episode: 121 meanReward: 9.4375 meanLoss: 0.3821 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1418 meanLossQtgt: 0.0952 meanLossQtgt_sigm: 0.1451
Episode: 122 meanReward: 9.4688 meanLoss: 0.3861 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1118 meanLossQtgt: 0.1473 meanLossQtgt_sigm: 0.1269
Episode: 123 meanReward: 9.5312 meanLoss: 0.3619 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0921 meanLossQtgt: 0.1519 meanLossQtgt_sigm: 0.1179

Episode: 175 meanReward: 9.7188 meanLoss: 0.5085 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0724 meanLossQtgt: 0.3247 meanLossQtgt_sigm: 0.1114
Episode: 176 meanReward: 9.7188 meanLoss: 0.5438 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0851 meanLossQtgt: 0.3403 meanLossQtgt_sigm: 0.1184
Episode: 177 meanReward: 9.6875 meanLoss: 0.6433 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1227 meanLossQtgt: 0.3828 meanLossQtgt_sigm: 0.1378
Episode: 178 meanReward: 9.6562 meanLoss: 0.7493 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1679 meanLossQtgt: 0.4176 meanLossQtgt_sigm: 0.1639
Episode: 179 meanReward: 9.7188 meanLoss: 0.7726 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1806 meanLossQtgt: 0.4220 meanLossQtgt_sigm: 0.1700
Episode: 180 meanReward: 9.7188 meanLoss: 0.9204 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1133 meanLossQtgt: 0.6712 meanLossQtgt_sigm: 0.1360
Episode: 181 meanReward: 9.6875 meanLoss: 1.0372 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0719 meanLossQtgt: 0.8439 meanLossQtgt_sigm: 0.1214

Episode: 234 meanReward: 9.0000 meanLoss: 0.5215 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1501 meanLossQtgt: 0.2129 meanLossQtgt_sigm: 0.1585
Episode: 235 meanReward: 9.0000 meanLoss: 0.5242 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.1477 meanLossQtgt: 0.2220 meanLossQtgt_sigm: 0.1545
Episode: 236 meanReward: 9.0312 meanLoss: 0.5683 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0952 meanLossQtgt: 0.3431 meanLossQtgt_sigm: 0.1300
Episode: 237 meanReward: 9.0312 meanLoss: 0.5954 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0814 meanLossQtgt: 0.3910 meanLossQtgt_sigm: 0.1229
Episode: 238 meanReward: 9.0625 meanLoss: 0.5603 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0914 meanLossQtgt: 0.3379 meanLossQtgt_sigm: 0.1310
Episode: 239 meanReward: 9.0000 meanLoss: 0.5134 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0960 meanLossQtgt: 0.2878 meanLossQtgt_sigm: 0.1296
Episode: 240 meanReward: 9.0625 meanLoss: 0.5156 meanLossQlbl: 0.0000 meanLossQlbl_sigm: 0.0822 meanLossQtgt: 0.3063 meanLossQtgt_sigm: 0.1271

...

Episode: 10188 meanReward: 11.3125 meanLoss: 36.0653 meanLossQlbl: 22.3177 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 12.3613 meanLossQtgt_sigm: 0.6931
Episode: 10189 meanReward: 11.5000 meanLoss: 34.0464 meanLossQlbl: 21.1857 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.4744 meanLossQtgt_sigm: 0.6931
Episode: 10190 meanReward: 11.4688 meanLoss: 33.4310 meanLossQlbl: 20.8474 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.1973 meanLossQtgt_sigm: 0.6931
Episode: 10191 meanReward: 11.4688 meanLoss: 31.2189 meanLossQlbl: 19.5555 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 10.2771 meanLossQtgt_sigm: 0.6931
Episode: 10192 meanReward: 11.4688 meanLoss: 25.5576 meanLossQlbl: 16.4132 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 7.7581 meanLossQtgt_sigm: 0.6931
Episode: 10193 meanReward: 11.5625 meanLoss: 32.1566 meanLossQlbl: 20.2435 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 10.5268 meanLossQtgt_sigm: 0.6931
Episode: 10194 meanReward: 11.7500 meanLoss: 35.9215 meanLossQlbl: 22.2066 meanLossQlbl_sigm: 0.6931 meanLo

Episode: 10244 meanReward: 10.6875 meanLoss: 35.5265 meanLossQlbl: 22.0116 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 12.1286 meanLossQtgt_sigm: 0.6931
Episode: 10245 meanReward: 10.7188 meanLoss: 33.2217 meanLossQlbl: 20.7879 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.0475 meanLossQtgt_sigm: 0.6931
Episode: 10246 meanReward: 10.8125 meanLoss: 33.6295 meanLossQlbl: 21.0322 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.2110 meanLossQtgt_sigm: 0.6931
Episode: 10247 meanReward: 10.8438 meanLoss: 33.9091 meanLossQlbl: 21.1604 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.3625 meanLossQtgt_sigm: 0.6931
Episode: 10248 meanReward: 11.0625 meanLoss: 34.0912 meanLossQlbl: 21.2058 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.4991 meanLossQtgt_sigm: 0.6931
Episode: 10249 meanReward: 11.0000 meanLoss: 33.1930 meanLossQlbl: 20.7139 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.0928 meanLossQtgt_sigm: 0.6931
Episode: 10250 meanReward: 11.0000 meanLoss: 33.2060 meanLossQlbl: 20.7668 meanLossQlbl_sigm: 0.6931 meanL

Episode: 10300 meanReward: 10.2500 meanLoss: 33.0852 meanLossQlbl: 20.7894 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 10.9095 meanLossQtgt_sigm: 0.6931
Episode: 10301 meanReward: 10.2812 meanLoss: 35.6530 meanLossQlbl: 22.1467 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 12.1200 meanLossQtgt_sigm: 0.6931
Episode: 10302 meanReward: 10.2812 meanLoss: 36.0627 meanLossQlbl: 22.3235 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 12.3529 meanLossQtgt_sigm: 0.6931
Episode: 10303 meanReward: 10.4688 meanLoss: 35.1820 meanLossQlbl: 21.7495 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 12.0462 meanLossQtgt_sigm: 0.6931
Episode: 10304 meanReward: 10.4375 meanLoss: 34.4258 meanLossQlbl: 21.2678 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.7717 meanLossQtgt_sigm: 0.6931
Episode: 10305 meanReward: 10.4375 meanLoss: 30.8837 meanLossQlbl: 19.3569 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 10.1405 meanLossQtgt_sigm: 0.6931
Episode: 10306 meanReward: 10.5312 meanLoss: 30.3325 meanLossQlbl: 19.1494 meanLossQlbl_sigm: 0.6931 meanL

Episode: 10356 meanReward: 10.7500 meanLoss: 35.6147 meanLossQlbl: 22.0795 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 12.1489 meanLossQtgt_sigm: 0.6931
Episode: 10357 meanReward: 10.5312 meanLoss: 34.6108 meanLossQlbl: 21.6959 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.5286 meanLossQtgt_sigm: 0.6931
Episode: 10358 meanReward: 10.5625 meanLoss: 31.8748 meanLossQlbl: 20.0867 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 10.4018 meanLossQtgt_sigm: 0.6931
Episode: 10359 meanReward: 10.5625 meanLoss: 34.3987 meanLossQlbl: 21.5696 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.4428 meanLossQtgt_sigm: 0.6931
Episode: 10360 meanReward: 10.5312 meanLoss: 35.0443 meanLossQlbl: 21.9868 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.6712 meanLossQtgt_sigm: 0.6931
Episode: 10361 meanReward: 10.5312 meanLoss: 34.5742 meanLossQlbl: 21.7174 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.4705 meanLossQtgt_sigm: 0.6931
Episode: 10362 meanReward: 10.4062 meanLoss: 34.3366 meanLossQlbl: 21.6211 meanLossQlbl_sigm: 0.6931 meanL

Episode: 10412 meanReward: 10.8125 meanLoss: 31.9797 meanLossQlbl: 20.0741 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 10.5193 meanLossQtgt_sigm: 0.6931
Episode: 10413 meanReward: 10.8125 meanLoss: 28.1479 meanLossQlbl: 17.9798 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 8.7818 meanLossQtgt_sigm: 0.6931
Episode: 10414 meanReward: 10.7188 meanLoss: 32.6927 meanLossQlbl: 20.5972 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 10.7092 meanLossQtgt_sigm: 0.6931
Episode: 10415 meanReward: 10.8125 meanLoss: 36.1989 meanLossQlbl: 22.3763 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 12.4364 meanLossQtgt_sigm: 0.6931
Episode: 10416 meanReward: 10.9062 meanLoss: 34.8690 meanLossQlbl: 21.6098 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.8730 meanLossQtgt_sigm: 0.6931
Episode: 10417 meanReward: 10.8750 meanLoss: 33.3107 meanLossQlbl: 20.7657 meanLossQlbl_sigm: 0.6931 meanLossQtgt: 11.1587 meanLossQtgt_sigm: 0.6931
Episode: 10418 meanReward: 10.8438 meanLoss: 34.0143 meanLossQlbl: 21.3296 meanLossQlbl_sigm: 0.6931 meanLo

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Average losses')

## Testing

Let's checkout how our trained agent plays the game.

In [184]:
import gym

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model-seq.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]
    state = env.reset()
    total_reward = 0
    while True:
        env.render()
        action_logits, initial_state = sess.run([model.actions_logits, model.final_state],
                                                feed_dict = {model.states: state.reshape([1, -1]), 
                                                             model.initial_state: initial_state})
        action = np.argmax(action_logits)
        state, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
print('total_reward:{}'.format(total_reward))
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward:120.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.